In [1]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA

from imblearn.over_sampling import SMOTE


In [2]:
import csv
import pandas as pd
import numpy as np
import scipy.io as sio
data = pd.read_csv('data_for_student_case-2.csv')

In [3]:
## check how many kinds of currency are there?
currencylist = data['currencycode'].unique()
print(currencylist)


['MXN' 'AUD' 'NZD' 'GBP' 'SEK']


In [4]:
def perday(df,DateName):

     df[DateName]=pd.to_datetime(df[DateName])
     df=df.sort_values(by=DateName)
     df[DateName]=df[DateName].dt.date
     return df

In [5]:
## currency convert from amount to amountEUR


#cardverificationresponsesupplied: did the shopper provide his 3 digit CVC/CVV2 code?
#cvcresponsecode: Validation result of the CVC/CVV2 code: 0 = Unknown, 1=Match, 2=No Match, 3-6=Not checked  

def transf(df):
    currencyconvert = {"AUD":0.680237,"MXN":0.0487438,"NZD":0.630313,"GBP":1.18285,"SEK":0.103558}
    amountEUR = []
    for i in df.index:
        if df['currencycode'][i] == "MXN":
            amountEUR.append(currencyconvert["MXN"]*df['amount'][i])
#         print(datacharge.loc[i,['currencycode']])
        if df['currencycode'][i] == "AUD":
            amountEUR.append(currencyconvert["AUD"]*df['amount'][i])

        if df['currencycode'][i] == "NZD":
            amountEUR.append(currencyconvert["NZD"]*df['amount'][i])

        if df['currencycode'][i] == "GBP":
            amountEUR.append(currencyconvert["GBP"]*df['amount'][i])

        if df['currencycode'][i] == "SEK":
            amountEUR.append(currencyconvert["SEK"]*df['amount'][i])
#amountEUR

    df=df.loc[:,['amount','bookingdate','simple_journal','cardverificationcodesupplied','cvcresponsecode']]
    dfcs=df[(df['simple_journal']=='Chargeback') | (df['simple_journal']=='Settled')]


#from datetime to date difference
    dfcs=perday(dfcs,'bookingdate')
    firstday=dfcs.loc[17903,'bookingdate']
    dfcs['bookingdate'] = dfcs['bookingdate']-firstday
    dfcs['bookingdate']=dfcs['bookingdate'] / np.timedelta64(1, 'D')

    dfcs.loc[dfcs['cardverificationcodesupplied']==False,'cardverificationcodesupplied']=0.0
    dfcs.loc[dfcs['cardverificationcodesupplied']==True,'cardverificationcodesupplied']=1.0
    dfcs=dfcs[(dfcs['cardverificationcodesupplied']==0) | (dfcs['cardverificationcodesupplied']==1)]
    dfcs.loc[:,'cardverificationcodesupplied']=pd.to_numeric(dfcs.loc[:,'cardverificationcodesupplied'])
    
    dfcs.loc[dfcs['simple_journal']=='Chargeback','simple_journal']=1.0
    dfcs.loc[dfcs['simple_journal']=='Settled','simple_journal']=0.0
    dfcs.loc[:,'simple_journal']=pd.to_numeric(dfcs.loc[:,'simple_journal'])

    dfcs.loc[:,'cvcresponsecode']=pd.to_numeric(dfcs.loc[:,'cvcresponsecode'])
    
    


    return dfcs

In [6]:
dfcs=transf(data)

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [7]:
dfcs.dtypes

amount                          float64
bookingdate                     float64
simple_journal                  float64
cardverificationcodesupplied    float64
cvcresponsecode                   int64
dtype: object

In [9]:
##-----seperate the data to features and labels
dffeatures = dfcs[['amount','bookingdate','cardverificationcodesupplied','cvcresponsecode']]
labels = dfcs['simple_journal']


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors

## test with UNSMOTED train set


TP, FP, FN, TN = 0, 0, 0, 0
x_array = np.array(dffeatures)
y_array = np.array(labels)
usx = x_array
usy = y_array
x_train, x_test, y_train, y_test = train_test_split(usx, usy, test_size = 0.2)#test_size: proportion of train/test data
clf = neighbors.KNeighborsClassifier(n_neighbors=3)
clf.fit(x_train, y_train)
y_predict = clf.predict(x_test)
for i in range(len(y_predict)):
    if y_test[i]==1 and y_predict[i]==1:
        TP += 1
    if y_test[i]==0 and y_predict[i]==1:
        FP += 1
    if y_test[i]==1 and y_predict[i]==0:
        FN += 1
    if y_test[i]==0 and y_predict[i]==0:
        TN += 1
print ('TP: '+ str(TP))
print ('FP: '+ str(FP))
print ('FN: '+ str(FN))
print ('TN: '+ str(TN))

# print (confusion_matrix(y_test, answear))
#watch out the element in confusion matrix
# precision, recall, thresholds = precision_recall_curve(y_test, y_predict)
# predict_proba = clf.predict_proba(x_test)#the probability of each smple labelled to positive or negative

TP: 24
FP: 2
FN: 36
TN: 44707


In [13]:
## test with SMOTED train set

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import     SMOTE 

TP, FP, FN, TN = 0, 0, 0, 0
x_array = np.array(dffeatures)
y_array = np.array(labels)
usx = x_array
usy = y_array
##------TO DO : split the data by sequential , "Chargeback" is only valid after its booking data
##------train on the past data, and test the future data

x_train, x_test, y_train, y_test = train_test_split(usx, usy, test_size = 0.2)#test_size: proportion of train/test data
clf = neighbors.KNeighborsClassifier(n_neighbors=3)

## -----do SMOTE on training data here, 
## -----remember : split the data first, then do SMOTE on train set. NEVER do test on modified set.

print('Original dataset shape {}'.format(Counter(y_train)))
sm = SMOTE()
X_res, y_res = sm.fit_sample(x_train, y_train)
print('Resampled dataset shape {}'.format(Counter(y_res)))
x_train = X_res
y_train = y_res



clf.fit(x_train, y_train)
y_predict = clf.predict(x_test)
for i in range(len(y_predict)):
    if y_test[i]==1 and y_predict[i]==1:
        TP += 1
    if y_test[i]==0 and y_predict[i]==1:
        FP += 1
    if y_test[i]==1 and y_predict[i]==0:
        FN += 1
    if y_test[i]==0 and y_predict[i]==0:
        TN += 1
print ('TP: '+ str(TP))
print ('FP: '+ str(FP))
print ('FN: '+ str(FN))
print ('TN: '+ str(TN))

Original dataset shape Counter({0.0: 178802, 1.0: 271})
Resampled dataset shape Counter({0.0: 178802, 1.0: 178802})
TP: 63
FP: 595
FN: 11
TN: 44100
